In [1]:
# Kubeflow Pipeline SDK
import kfp
import kfp.dsl as dsl
import kfp.components as components
from kfp.components import func_to_container_op, InputPath, OutputPath
from typing import NamedTuple

In [2]:
def main(log_folder:str):
    import subprocess
    # 在虛擬容器中開啟sklearnserver，port為8001，模型為svm
    subprocess.run(["python", "-m", "sklearnserver", "--http_port", "8081", "--model_dir", "sklearnserver/sklearnserver/example_models/pkl/model"])


In [3]:
# Pipeline名稱
@dsl.pipeline(
    name='kserve-svm',
    description='kserve'
)

def kfservingPipeline():
    
    log_folder = '/information'
    pvc_name = "kserve-svm"

    # docker image
    image = "dfm871002/kserve-svm:1.0.0"
    
    # 定義pipeline名稱以及對應的function
    vop = dsl.VolumeOp(
        name=pvc_name,
        resource_name="kserve-test",
        size="1Gi",
        modes=dsl.VOLUME_MODE_RWM
    )
    
    main_op = func_to_container_op(
          func = main,
          base_image = image
    )
    
    #定義pipeline中每個work的順序關係
    main_task = main_op(log_folder).add_pvolumes({log_folder:vop.volume,})

In [4]:
# 編譯Pipeline
kfp.compiler.Compiler().compile(kfservingPipeline, 'sklearn.zip')